In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Make a function which converts the GDC mission time stamps to fully specified dates and times

In [2]:

# Example date format from GDC Ephemeris
exdatestr = ' 91/00:00:00.000'

def mission_elapsed_to_datetime(mission_elapsed_time_str):
    """
    Time stamps are assumed specified as follows:
    DOY2028/HH:MM:SS.000
    Where DOY2028 is the number of days since 2028/1/1
    Mission starts on DOY2028 = 91, April 1, 2028
    """
    
    doy2028str,tstr = mission_elapsed_time_str.split('/')
    doy2028 = int(doy2028str)
    
    # All timestamps appear to end with .000 (the data is at whole second cadence)
    # so we will hard code the .000 as part of the format
    tfmt = '%H:%M:%S.000'
    
    #Read in as a Python time
    time = datetime.datetime.strptime(tstr,tfmt).time()
    
    date = datetime.date(2028,1,1)+datetime.timedelta(days=doy2028)
    return pd.to_datetime(datetime.datetime.combine(date,time))
    
print(mission_elapsed_to_datetime(exdatestr))

2028-04-01 00:00:00


# Read the text file using Pandas

In [3]:
gdc_input_fn = 'GDC_Ephemeris_G1_Phase1a.txt'
df = pd.read_csv(gdc_input_fn,
                sep='\s+', #Any number of spaces is the seperator
                converters={0:mission_elapsed_to_datetime},#Convert the mission elapsed time format (column 0) to a fully specified date & time
                skiprows=6,
                names = ['time','x','y','z'],
                usecols = [0,1,2,3], #Only read the first 4 columns (ECI J2000 position)
                index_col=0 #Make the dataframe index by time
                )

In [4]:
df.head()

,x,y,z
time,,,
2028-04-01 00:00:00,-5390.990,-1724.765,-3664.336
2028-04-01 00:00:30,-5257.535,-1727.603,-3852.299
2028-04-01 00:01:00,-5117.934,-1728.422,-4035.746
2028-04-01 00:01:30,-4972.352,-1727.221,-4214.463
2028-04-01 00:02:00,-4820.961,-1724.002,-4388.242


# Use open source libraries (geospacepy-lite and apexpy) to convert coordinates
Find more information about these tools below:
* [geospacepy-lite](http://github.com/lkilcommons/geospacepy-lite)
* [apexpy](https://github.com/aburrell/apexpy)

In [5]:
from apexpy import Apex
apex = Apex()
apex.set_epoch(2028.4) #Date for IGRF evaluationt
apex.set_refh(110) #Reference altitude for Apex coordinates is 110 kilometers

In [6]:
from geospacepy.terrestrial_spherical import eci2ecef
from geospacepy.terrestrial_ellipsoidal import ecef_cart2geodetic
from geospacepy.special_datetime import datetimearr2jd

def datetime64_to_datetime(dt64):
    """black magic to convert a datetime64 (date format used by pandas) to a Python datetime"""
    return dt64.astype('M8[ms]').astype('O')

In [7]:
dts = [datetime64_to_datetime(dt64) for dt64 in df.index.values] # Make list of Python datetimes from Pandas datetimes
jds = datetimearr2jd(dts) #Convert to Julian dates (format required for ECI to ECEF conversion)
#Convert ECI to geodetic, and then to Apex
R_ECI = np.column_stack([df['x'].values,df['y'].values,df['z'].values])
R_ECEF = eci2ecef(R_ECI,jds)
gdlat,glon,alt_m = ecef_cart2geodetic(R_ECEF*1000.) #Expects/returns units of meters
alt_km = alt_m/1000.
alat,alon = apex.geo2apex(gdlat,glon,alt_km) #Expects altitude in km

#Calculate magnetic local time of each spacecraft position
mlt = np.full_like(alat,np.nan)
for i in range(alat.size):
    mlt[i]=apex.mlon2mlt(alon[i],dts[i])

#Add geodetic and apex locations to dataframe
df['gdlat']=gdlat
df['glon']=glon
df['alt_km']=alt_km
df['apex110lat']=alat
df['apex110lon']=alon
df['mlt']=mlt

# Write out the magnetic coordinates positions using Pandas

In [11]:
import os
gdc_output_fn = os.path.splitext(gdc_input_fn)[0]+'_Apex.csv'
df.to_csv(gdc_output_fn)